In [1]:
%%html
<style>
.cm-s-ipython .CodeMirror-matchingbracket { color: LimeGreen !important;}
</style>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import pprint
from scipy import stats
import random
from pandas.tools.plotting import table

In [6]:
import pathlib

In [3]:
from sklearn import preprocessing

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Functions

In [6]:
def pre_process_df(df):
    
    #Drop unwanted columns
    df_dropped = df.drop(columns=['Unnamed: 0','item_name',
                                  'stash_id','stash_feed',
                                  '_id',
                                  'date_month','date_year','date',
                                  'league','rarity',
                                  'price_currency','price_raw',
                                  'time_minutes','time'])
    
    # delete columns with no values in them, which means only zero
    proc_df = df_dropped.loc[:,(df!=0).any (axis=0)]
    
    # Clean up of data. 
    ## Prices were imported as 'Object' not 'float'. We need to convert them to float.
    proc_df[['price_amount']] = pd.to_numeric(proc_df['price_amount'],errors='coerce')
    
    # Remove rows where price_amount is NaN or zero
    proc_df = proc_df[pd.notnull(proc_df['price_amount'])]
    proc_df = proc_df.loc[proc_df['price_amount']!=0]
    
    return proc_df

def compute_corr(df,method='kendall',filename=''):
    
    #min_periods = int(len(df))*0.1
    cols = list(df.filter(regex='(Attacks per Second|Energy Shield|Elemental Damage|Critical Strike Chance|Physical Damage|influence|Armour|sockets_number|linked_sockets|Evasion Rating|item_category)|(?=^co_|ex_|im_|en_$)(^.*$)').columns.values)
    df[cols] = df[cols].replace({0:np.nan, 0.0:np.nan})
    #df[df.filter(regex='(?=^co_|ex_|im_|en_$)(^.*$)') <= 0.0] = np.nan
    corr = df.corr(method)
    corr = corr.dropna('columns',how='all')
    corr = corr.dropna('rows',how='all')
    df = df.fillna(0)
    
    return corr

def remove_outliers_IQR(item_dataframe,column_label = 'price_amount',high_quantile=0.75):
    '''Function removes outliers from a dataframe along the price_amount column by default.

    Input:
        df: pandas DataFrame
        column_label: along which column to check for outliers(default = 'price_amount')
        high_quantile: high-end quantile to use on boxplot'''
    
    Q1 = item_dataframe[column_label].quantile(1-high_quantile)
    Q3 = item_dataframe[column_label].quantile(high_quantile)
    IQR = Q3 - Q1
    new_df = item_dataframe[~((item_dataframe[column_label] < (Q1 - 1.5 * IQR))|(item_dataframe[column_label] > (Q3 + 1.5 * IQR)))]
    
    return new_df

def remove_outliers_zscore(item_dataframe,column_labels = ['price_amount'],threshold=3,show_results=False):
    '''Function removes outliers using z-score from a dataframe along the price_amount column by default.

    Input:
        df: pandas DataFrame
        column_label: along which columns to check for outliers(default = ['price_amount'])
        show_results: show results before and after removing outliers(default = False)
        size: vertical and horizontal size of the plot'''
    
    z_score = np.abs(stats.zscore(item_dataframe[column_labels]))
    new_df = item_dataframe[(z_score < threshold)]
    if show_results:
        data_outliers_index = np.where(z_score > threshold)[0]
        print('Data outliers for "{}":'.format(item_dataframe['item_name'][0]))
        for id in data_outliers_index:
              print('index: {:<10d}{}: {:<10f}'.format(id,column_labels[0],item_dataframe.iloc[id][column_labels[0]]))
        print('Removed {} rows'.format(item_dataframe.shape[0]-new_df.shape[0]))
    return new_df


def produce_decision_dataframe(item_df,correlations_df=pd.DataFrame,incl_outliers=True,method='z-score',threshold=2,quantile=0.8):
    
    d_df = item_df
    
    if not incl_outliers:
        if method == 'z-score' : 
            d_df = remove_outliers_zscore(d_df,threshold=threshold)
        elif method == 'IQR' :
            d_df = remove_outliers_IQR(d_df,high_quantile=quantile)
        else:
            raise Exception('\t\tWrong outlier mode. Valid options mode = [z-score | IQR]')
    
    if correlations_df.empty:
        columns = ['item_name','feature','corr_value','no_features','transactions','std']
        correlations_df =  pd.DataFrame(columns=columns)
    
    corr=compute_corr(d_df,method='kendall')
    #corr_filtered = corr['price_amount'].filter(regex='(item_category|corrupted|Attacks per Second|Energy Shield|Elemental Damage|Critical Strike Chance|Physical Damage|influence|Armour|sockets_number|linked_sockets|Quality|Evasion Rating)|(?=^co_|ex_|im_|en_$)(^.*$)').drop(labels=['ex_conv_rate'],axis=0).dropna()
    corr_filtered = corr['price_amount'].filter(regex='(date_day|item_category|corrupted|Attacks per Second|Energy Shield|Elemental Damage|Critical Strike Chance|Physical Damage|influence|Armour|sockets_number|linked_sockets|Quality|Evasion Rating)|(?=^co_|ex_|im_|en_$)(^.*$)').dropna()
    for row in corr_filtered.index:
        correlations_df = correlations_df.append({'item_name':d_df['item_name'].unique()[0],
                                'feature':row,
                                'corr_value': corr_filtered[row],
                                'no_features':len(corr_filtered),
                                'transactions':d_df.groupby('item_name')['item_name'].count().values[0],
                                'std':d_df['price_amount'].describe()['std']},ignore_index=True)
    
    return correlations_df

def produce_corr_based_df(df_per_item_name,method='z-score',threshold=2,quantile=0.8):
    
    columns = ['item_name','feature','corr_value','no_features','transactions','std']

    df =  pd.DataFrame(columns=columns)

    count = 0
    for dataF in df_per_item_name:
        count= count+1
        if count%200==0:
            print("Processed {} item_names".format(count))
        item_df = df_per_item_name[dataF]
        df = produce_decision_dataframe(item_df,df,incl_outliers=False,method=method,threshold=threshold,quantile=quantile)
        
    return df

def filter_decision_df(df, days=7, min_corr=0.1, min_no_features=2, min_std=5.0):
    
    min_trx = days*24
    
    df_filtered = df[(abs(df['corr_value'])>=min_corr) & \
                                          (df['transactions'] > min_trx) & \
                                          (df['std'] > min_std)]
    
    df_filtered['no_features'] = df_filtered.groupby('item_name')['item_name'].transform('count')
    df_filtered = df_filtered[df_filtered['no_features'] >= min_no_features]
    
    return df_filtered

def convert_column_values_string_to_rankInt(df) -> pd.DataFrame:
    for column in df.columns:
        if df[column].dtype == type(object):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

    return df

# Processing all the rare item categories

In [11]:
directory = "C:/Users/Digi/Desktop/poe_price_predictor/record_creator/record_creator/csv/rare/"
files_to_process = [x for x in glob.glob(directory+'*_values.csv')]

df_per_category = {}

for f in files_to_process:
    
    print("Processing start : {}".format(f[81:]))
    
    df = pd.read_csv(f)
    
    proc_category_df = pre_process_df(df)
        
    print("Processing end : {}".format(f[81:]))
    
    category=proc_category_df['item_category'].unique()[0]
    df_per_category[category] = proc_category_df

Processing start : wand_values.csv
Processing end : wand_values.csv


In [12]:
for key,category_df in df_per_category.items():
    "{}_columns = {}".format(key,category_df.shape[1])
    run_all_statistics(category_df)

'wand_columns = 126'

Correlations with outliers
Removing outliers zscore
Removing outliers IQR
Correlations without outliers zscore
Correlations without outliers IQR


# Statistics

## With outliers

### Data percentage present per column

In [113]:
((proc_category_df!=0).sum()/len(proc_category_df)*100)

date_day                                               100.000000
ex_conv_rate                                           100.000000
item_category                                          100.000000
item_level                                             100.000000
socket_colors                                          100.000000
price_amount                                            99.766039
time_hours                                              90.280516
days_in_snapshot                                        64.917302
ex_# to maximum life                                    39.551713
ex_# to maximum mana                                    22.985994
ex_# to maximum energy shield                           22.621790
ex_adds # physical damage to attacks                    22.201373
ex_#% to lightning resistance                           19.451240
ex_#% to fire resistance                                19.186005
ex_#% to cold resistance                                18.885537
ex_#% to g

### Compute correlations for different methods

In [134]:
corr_kendall = compute_corr(proc_category_df,method='kendall')
corr_spear = compute_corr(proc_category_df,method='spearman')
corr_pears = compute_corr(proc_category_df,method='pearson')

In [135]:
corr_spear['price_amount'].sort_values(ascending=False)
(corr_spear['price_amount']>0).sum()
(corr_kendall['price_amount']>0).sum()
(corr_pears['price_amount']>0).sum()

price_amount                                                        1.000000
ex_minions have #% increased movement speed                         0.662454
ex_#% increased attack speed                                        0.630827
ex_#% increased quantity of items found                             0.593084
ex_#% of life regenerated per second                                0.501488
co_#% increased quantity of items found                             0.418741
item_level                                                          0.358838
ex_damage penetrates #% elemental resistances                       0.342013
ex_# to maximum life                                                0.320331
ex_#% increased movement speed                                      0.320085
ex_gain #% of physical damage as extra lightning damage             0.308968
ex_#% increased elemental damage with attack skills                 0.294738
ex_#% to global critical strike multiplier                          0.280317

99

101

92

### Basic describe and put all statistics in a dataframe

In [9]:
def run_all_statistics(df):
    item_category = df['item_category'].unique()[0]
    
    print("Correlations with outliers")
    corr_kendall = compute_corr(df,method='kendall')
    corr_pears = compute_corr(df,method='pearson')
    corr_spear = compute_corr(df,method='spearman')
    column_statistics_o = df.describe().transpose()
    column_statistics_o['median'] = df.median()
    column_statistics_o['variance'] = df.var()
    column_statistics_o['count'] = (df.notna()).sum()
    column_statistics_o['corr_kend_price'] = corr_kendall['price_amount'].round(3)
    column_statistics_o['corr_pears_price'] =corr_pears['price_amount'].round(3)
    column_statistics_o['corr_spear_price'] =corr_spear['price_amount'].round(3)
    column_statistics_o['percentage_to_rows'] = (df.notna()).sum()/len(df)*100
    column_statistics_o.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_o.csv'.format(item_category),sep='^')
    
    print("Removing outliers zscore")
    df_zscore_1 = remove_outliers_zscore(df,threshold=1)
    df_zscore_2 = remove_outliers_zscore(df,threshold=2)
    df_zscore_3 = remove_outliers_zscore(df,threshold=3)

    print("Removing outliers IQR")
    df_IQR_70 = remove_outliers_IQR(df,high_quantile=0.7)
    df_IQR_80 = remove_outliers_IQR(df,high_quantile=0.8)
    df_IQR_90 = remove_outliers_IQR(df,high_quantile=0.9)
    
    print("Correlations without outliers zscore")
    corr_kendall_zscore_2 = compute_corr(df_zscore_2,method='kendall')
    corr_spear_zscore_2 = compute_corr(df_zscore_2,method='spearman')
    corr_pears_zscore_2 = compute_corr(df_zscore_2,method='pearson')

    column_statistics_zscore_2 = df_zscore_2.describe().transpose()
    column_statistics_zscore_2['median'] = df_zscore_2.median()
    column_statistics_zscore_2['variance'] = df_zscore_2.var()
    column_statistics_zscore_2['count'] = (df_zscore_2.notna()).sum()
    column_statistics_zscore_2['corr_kend_price'] = corr_kendall_zscore_2['price_amount'].round(3)
    column_statistics_zscore_2['corr_pears_price'] =corr_pears_zscore_2['price_amount'].round(3)
    column_statistics_zscore_2['corr_spear_price'] =corr_spear_zscore_2['price_amount'].round(3)
    column_statistics_zscore_2['percentage_to_rows'] = (df_zscore_2.notna()).sum()/len(df_zscore_2)*100
    column_statistics_zscore_2.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_zscore_2.csv'.format(item_category),sep='^')

    corr_kendall_zscore_1 = compute_corr(df_zscore_1,method='kendall')
    corr_spear_zscore_1 = compute_corr(df_zscore_1,method='spearman')
    corr_pears_zscore_1 = compute_corr(df_zscore_1,method='pearson')

    column_statistics_zscore_1 = df_zscore_1.describe().transpose()
    column_statistics_zscore_1['median'] = df_zscore_1.median()
    column_statistics_zscore_1['variance'] = df_zscore_1.var()
    column_statistics_zscore_1['count'] = (df_zscore_1.notna()).sum()
    column_statistics_zscore_1['corr_kend_price'] = corr_kendall_zscore_1['price_amount'].round(3)
    column_statistics_zscore_1['corr_pears_price'] =corr_pears_zscore_1['price_amount'].round(3)
    column_statistics_zscore_1['corr_spear_price'] =corr_spear_zscore_1['price_amount'].round(3)
    column_statistics_zscore_1['percentage_to_rows'] = (df_zscore_1.notna()).sum()/len(df_zscore_1)*100
    column_statistics_zscore_1.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_zscore_1.csv'.format(item_category),sep='^')
    
    corr_kendall_zscore_3 = compute_corr(df_zscore_3,method='kendall')
    corr_spear_zscore_3 = compute_corr(df_zscore_3,method='spearman')
    corr_pears_zscore_3 = compute_corr(df_zscore_3,method='pearson')

    column_statistics_zscore_3 = df_zscore_3.describe().transpose()
    column_statistics_zscore_3['median'] = df_zscore_3.median()
    column_statistics_zscore_3['variance'] = df_zscore_3.var()
    column_statistics_zscore_3['count'] = (df_zscore_3.notna()).sum()
    column_statistics_zscore_3['corr_kend_price'] = corr_kendall_zscore_3['price_amount'].round(3)
    column_statistics_zscore_3['corr_pears_price'] =corr_pears_zscore_3['price_amount'].round(3)
    column_statistics_zscore_3['corr_spear_price'] =corr_spear_zscore_3['price_amount'].round(3)
    column_statistics_zscore_3['percentage_to_rows'] = (df_zscore_3.notna()).sum()/len(df_zscore_3)*100
    column_statistics_zscore_3.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_zscore_3.csv'.format(item_category),sep='^')

    print("Correlations without outliers IQR")
    corr_kendall_IQR_70 = compute_corr(df_IQR_70,method='kendall')
    corr_spear_IQR_70 = compute_corr(df_IQR_70,method='spearman')
    corr_pears_IQR_70 = compute_corr(df_IQR_70,method='pearson')

    column_statistics_IQR_70 = df_IQR_70.describe().transpose()
    column_statistics_IQR_70['median'] = df_IQR_70.median()
    column_statistics_IQR_70['variance'] = df_IQR_70.var()
    column_statistics_IQR_70['count'] = (df_IQR_70.notna()).sum()
    column_statistics_IQR_70['corr_kend_price'] = corr_kendall_IQR_70['price_amount'].round(3)
    column_statistics_IQR_70['corr_pears_price'] =corr_pears_IQR_70['price_amount'].round(3)
    column_statistics_IQR_70['corr_spear_price'] =corr_spear_IQR_70['price_amount'].round(3)
    column_statistics_IQR_70['percentage_to_rows'] = (df_IQR_70.notna()).sum()/len(df_IQR_70)*100
    column_statistics_IQR_70.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_IQR_70.csv'.format(item_category),sep='^')

    corr_kendall_IQR_80 = compute_corr(df_IQR_80,method='kendall')
    corr_spear_IQR_80 = compute_corr(df_IQR_80,method='spearman')
    corr_pears_IQR_80 = compute_corr(df_IQR_80,method='pearson')

    column_statistics_IQR_80 = df_IQR_80.describe().transpose()
    column_statistics_IQR_80['median'] = df_IQR_80.median()
    column_statistics_IQR_80['variance'] = df_IQR_80.var()
    column_statistics_IQR_80['count'] = (df_IQR_80.notna()).sum()
    column_statistics_IQR_80['corr_kend_price'] = corr_kendall_IQR_80['price_amount'].round(3)
    column_statistics_IQR_80['corr_pears_price'] =corr_pears_IQR_80['price_amount'].round(3)
    column_statistics_IQR_80['corr_spear_price'] =corr_spear_IQR_80['price_amount'].round(3)
    column_statistics_IQR_80['percentage_to_rows'] = (df_IQR_80.notna()).sum()/len(df_IQR_80)*100
    column_statistics_IQR_80.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_IQR_80.csv'.format(item_category),sep='^')

    corr_kendall_IQR_90 = compute_corr(df_IQR_90,method='kendall')
    corr_spear_IQR_90 = compute_corr(df_IQR_90,method='spearman')
    corr_pears_IQR_90 = compute_corr(df_IQR_90,method='pearson')

    column_statistics_IQR_90 = df_IQR_90.describe().transpose()
    column_statistics_IQR_90['median'] = df_IQR_90.median()
    column_statistics_IQR_90['variance'] = df_IQR_90.var()
    column_statistics_IQR_90['count'] = (df_IQR_90.notna()).sum()
    column_statistics_IQR_90['corr_kend_price'] = corr_kendall_IQR_90['price_amount'].round(3)
    column_statistics_IQR_90['corr_pears_price'] =corr_pears_IQR_90['price_amount'].round(3)
    column_statistics_IQR_90['corr_spear_price'] =corr_spear_IQR_90['price_amount'].round(3)
    column_statistics_IQR_90['percentage_to_rows'] = (df_IQR_90.notna()).sum()/len(df_IQR_90)*100
    column_statistics_IQR_90.round(3).to_csv('C:/Users/Digi/Desktop/rares/{}_stats_IQR_90.csv'.format(item_category),sep='^')

In [168]:
column_statistics_o = proc_category_df.describe().transpose()
column_statistics_o['median'] = proc_category_df.median()
column_statistics_o['variance'] = proc_category_df.var()
column_statistics_o['count'] = (proc_category_df.notna()).sum()
column_statistics_o['corr_kend_price'] = corr_kendall['price_amount'].round(3)
column_statistics_o['corr_pears_price'] =corr_pears['price_amount'].round(3)
column_statistics_o['corr_spear_price'] =corr_spear['price_amount'].round(3)
column_statistics_o['percentage_to_rows'] = (proc_category_df.notna()).sum()/len(proc_category_df)*100

In [179]:
pd.set_option('display.max_rows',171)
column_statistics_o

,count,mean,std,min,25%,50%,75%,max,median,variance,corr_kend_price,corr_pears_price,corr_spear_price,percentage_to_rows
co_# life regenerated per second,1052,3.025665,5.965545e-01,2.000000,2.50,3.00,3.500,4.00000,3.00,3.558773e-01,-0.013,-0.002,-0.018,0.417435
co_# to all attributes,1125,13.179556,2.005271e+00,10.000000,11.00,13.00,15.000,16.00000,13.00,4.021112e+00,-0.011,0.028,-0.015,0.446402
co_# to dexterity,926,25.091793,3.186322e+00,20.000000,22.00,25.00,28.000,30.00000,25.00,1.015265e+01,-0.003,-0.002,-0.005,0.367438
co_# to dexterity and intelligence,949,20.182297,2.585961e+00,16.000000,18.00,20.00,22.000,24.00000,20.00,6.687197e+00,0.038,0.098,0.051,0.376565
co_# to intelligence,1070,25.165421,3.251210e+00,20.000000,22.00,25.00,28.000,30.00000,25.00,1.057037e+01,0.013,-0.045,0.018,0.424578
co_# to maximum number of zombies,797,1.000000,0.000000e+00,1.000000,1.00,1.00,1.000,1.00000,1.00,0.000000e+00,NaN,NaN,NaN,0.316251
co_# to strength,1004,25.135458,3.103335e+00,20.000000,22.00,25.00,28.000,30.00000,25.00,9.630686e+00,0.016,0.052,0.022,0.398389
co_# to strength and dexterity,962,20.019751,2.545450e+00,16.000000,18.00,20.00,22.000,24.00000,20.00,6.479318e+00,0.013,-0.028,0.017,0.381723
co_# to strength and intelligence,1053,20.155745,2.583300e+00,16.000000,18.00,20.00,22.000,24.00000,20.00,6.673439e+00,0.009,0.025,0.013,0.417832
co_#% additional physical damage reduction,290,5.017241,8.212443e-01,4.000000,4.00,5.00,6.000,6.00000,5.00,6.744422e-01,0.015,0.058,0.019,0.115073


In [170]:
column_statistics_o.to_csv('astatistics_with_outliers.csv',sep='^')

## Without outliers

In [8]:
proc_category_df_zscore_1 = remove_outliers_zscore(proc_category_df,threshold=1)
proc_category_df_zscore_2 = remove_outliers_zscore(proc_category_df,threshold=2)
proc_category_df_zscore_3 = remove_outliers_zscore(proc_category_df,threshold=3)

proc_category_df_IQR_70 = remove_outliers_IQR(proc_category_df,high_quantile=0.7)
proc_category_df_IQR_80 = remove_outliers_IQR(proc_category_df,high_quantile=0.8)
proc_category_df_IQR_90 = remove_outliers_IQR(proc_category_df,high_quantile=0.9)

### Zscore , threshold = 1

In [9]:
corr_kendall_zscore_1 = compute_corr(proc_category_df_zscore_1,method='kendall')
corr_spear_zscore_1 = compute_corr(proc_category_df_zscore_1,method='spearman')
corr_pears_zscore_1 = compute_corr(proc_category_df_zscore_1,method='pearson')

column_statistics_zscore_1 = proc_category_df_zscore_1.describe().transpose()
column_statistics_zscore_1['median'] = proc_category_df_zscore_1.median()
column_statistics_zscore_1['variance'] = proc_category_df_zscore_1.var()
column_statistics_zscore_1['count'] = (proc_category_df_zscore_1.notna()).sum()
column_statistics_zscore_1['corr_kend_price'] = corr_kendall_zscore_1['price_amount'].round(3)
column_statistics_zscore_1['corr_pears_price'] =corr_pears_zscore_1['price_amount'].round(3)
column_statistics_zscore_1['corr_spear_price'] =corr_spear_zscore_1['price_amount'].round(3)
column_statistics_zscore_1['percentage_to_rows'] = (proc_category_df_zscore_1.notna()).sum()/len(proc_category_df_zscore_1)*100
column_statistics_zscore_1.to_csv('C:/Users/Digi/Desktop/rares/amulet_stats_zscore_1.csv',sep='^')

### Zscore , threshold = 2

In [10]:
corr_kendall_zscore_2 = compute_corr(proc_category_df_zscore_2,method='kendall')
corr_spear_zscore_2 = compute_corr(proc_category_df_zscore_2,method='spearman')
corr_pears_zscore_2 = compute_corr(proc_category_df_zscore_2,method='pearson')

column_statistics_zscore_2 = proc_category_df_zscore_2.describe().transpose()
column_statistics_zscore_2['median'] = proc_category_df_zscore_2.median()
column_statistics_zscore_2['variance'] = proc_category_df_zscore_2.var()
column_statistics_zscore_2['count'] = (proc_category_df_zscore_2.notna()).sum()
column_statistics_zscore_2['corr_kend_price'] = corr_kendall_zscore_2['price_amount'].round(3)
column_statistics_zscore_2['corr_pears_price'] =corr_pears_zscore_2['price_amount'].round(3)
column_statistics_zscore_2['corr_spear_price'] =corr_spear_zscore_2['price_amount'].round(3)
column_statistics_zscore_2['percentage_to_rows'] = (proc_category_df_zscore_2.notna()).sum()/len(proc_category_df_zscore_2)*100
column_statistics_zscore_2.to_csv('amulet_stats_zscore_2.csv',sep='^')

### Zscore , threshold = 3

In [11]:
corr_kendall_zscore_3 = compute_corr(proc_category_df_zscore_3,method='kendall')
corr_spear_zscore_3 = compute_corr(proc_category_df_zscore_3,method='spearman')
corr_pears_zscore_3 = compute_corr(proc_category_df_zscore_3,method='pearson')

column_statistics_zscore_3 = proc_category_df_zscore_3.describe().transpose()
column_statistics_zscore_3['median'] = proc_category_df_zscore_3.median()
column_statistics_zscore_3['variance'] = proc_category_df_zscore_3.var()
column_statistics_zscore_3['count'] = (proc_category_df_zscore_3.notna()).sum()
column_statistics_zscore_3['corr_kend_price'] = corr_kendall_zscore_3['price_amount'].round(3)
column_statistics_zscore_3['corr_pears_price'] =corr_pears_zscore_3['price_amount'].round(3)
column_statistics_zscore_3['corr_spear_price'] =corr_spear_zscore_3['price_amount'].round(3)
column_statistics_zscore_3['percentage_to_rows'] = (proc_category_df_zscore_3.notna()).sum()/len(proc_category_df_zscore_3)*100
column_statistics_zscore_3.to_csv('amulet_stats_zscore_3.csv',sep='^')

### IQR , high_quantile = 0.7

In [12]:
corr_kendall_IQR_70 = compute_corr(proc_category_df_IQR_70,method='kendall')
corr_spear_IQR_70 = compute_corr(proc_category_df_IQR_70,method='spearman')
corr_pears_IQR_70 = compute_corr(proc_category_df_IQR_70,method='pearson')

column_statistics_IQR_70 = proc_category_df_IQR_70.describe().transpose()
column_statistics_IQR_70['median'] = proc_category_df_IQR_70.median()
column_statistics_IQR_70['variance'] = proc_category_df_IQR_70.var()
column_statistics_IQR_70['count'] = (proc_category_df_IQR_70.notna()).sum()
column_statistics_IQR_70['corr_kend_price'] = corr_kendall_IQR_70['price_amount'].round(3)
column_statistics_IQR_70['corr_pears_price'] =corr_pears_IQR_70['price_amount'].round(3)
column_statistics_IQR_70['corr_spear_price'] =corr_spear_IQR_70['price_amount'].round(3)
column_statistics_IQR_70['percentage_to_rows'] = (proc_category_df_IQR_70.notna()).sum()/len(proc_category_df_IQR_70)*100
column_statistics_IQR_70.to_csv('amulet_stats_IQR_70.csv',sep='^')


### IQR , high_quantile = 0.8

In [13]:
corr_kendall_IQR_80 = compute_corr(proc_category_df_IQR_80,method='kendall')
corr_spear_IQR_80 = compute_corr(proc_category_df_IQR_80,method='spearman')
corr_pears_IQR_80 = compute_corr(proc_category_df_IQR_80,method='pearson')

column_statistics_IQR_80 = proc_category_df_IQR_80.describe().transpose()
column_statistics_IQR_80['median'] = proc_category_df_IQR_80.median()
column_statistics_IQR_80['variance'] = proc_category_df_IQR_80.var()
column_statistics_IQR_80['count'] = (proc_category_df_IQR_80.notna()).sum()
column_statistics_IQR_80['corr_kend_price'] = corr_kendall_IQR_80['price_amount'].round(3)
column_statistics_IQR_80['corr_pears_price'] =corr_pears_IQR_80['price_amount'].round(3)
column_statistics_IQR_80['corr_spear_price'] =corr_spear_IQR_80['price_amount'].round(3)
column_statistics_IQR_80['percentage_to_rows'] = (proc_category_df_IQR_80.notna()).sum()/len(proc_category_df_IQR_80)*100
column_statistics_IQR_80.to_csv('amulet_stats_IQR_80.csv',sep='^')

### IQR , high_quantile = 0.9

In [14]:

corr_kendall_IQR_90 = compute_corr(proc_category_df_IQR_90,method='kendall')
corr_spear_IQR_90 = compute_corr(proc_category_df_IQR_90,method='spearman')
corr_pears_IQR_90 = compute_corr(proc_category_df_IQR_90,method='pearson')

column_statistics_IQR_90 = proc_category_df_IQR_90.describe().transpose()
column_statistics_IQR_90['median'] = proc_category_df_IQR_90.median()
column_statistics_IQR_90['variance'] = proc_category_df_IQR_90.var()
column_statistics_IQR_90['count'] = (proc_category_df_IQR_90.notna()).sum()
column_statistics_IQR_90['corr_kend_price'] = corr_kendall_IQR_90['price_amount'].round(3)
column_statistics_IQR_90['corr_pears_price'] =corr_pears_IQR_90['price_amount'].round(3)
column_statistics_IQR_90['corr_spear_price'] =corr_spear_IQR_90['price_amount'].round(3)
column_statistics_IQR_90['percentage_to_rows'] = (proc_category_df_IQR_90.notna()).sum()/len(proc_category_df_IQR_90)*100
column_statistics_IQR_90.to_csv('amulet_stats_IQR_90.csv',sep='^')

# Comparing and filtering

In [59]:
directory = "C:/Users/Digi/Desktop/rares/"
files_to_process = [x for x in glob.glob(directory+''+'*.csv')]

dfs_per_category = {}

for f in files_to_process:
    base_fname = pathlib.PurePath(f).name
    category = base_fname.split('_')[0]
    type_of_csv = base_fname.split('_',maxsplit=1)[1]
    if category not in dfs_per_category:
        dfs_per_category[category] = {}
        
    df = pd.read_csv(f,delimiter='^').set_index('Unnamed: 0')
    
    if type_of_csv not in dfs_per_category[category]:
        dfs_per_category[category][type_of_csv] = df
    

In [77]:
columns = ['item_category','stats_o.csv','stats_zscore_1.csv','stats_zscore_2.csv','stats_zscore_3.csv','stats_IQR_70.csv','stats_IQR_80.csv','stats_IQR_90.csv']

compare_categories_df =  pd.DataFrame(columns=columns)

for key_item_category, type_of_df in dfs_per_category.items():
    
    category_dict = {'item_category':key_item_category}
    
    for df_category,df in type_of_df.items():
        category_dict[df_category] =df.loc['price_amount','max']
    compare_categories_df = compare_categories_df.append(category_dict,ignore_index=True)

compare_categories_df = compare_categories_df.set_index('item_category')

In [78]:
compare_categories_df.columns = ['max_price_outliers','max_price_zscore_1','max_price_zscore_2','max_price_zscore_3','max_price_IQR_70','max_price_IQR_80','max_price_IQR_90']

In [79]:
compare_categories_df.astype(int)

,max_price_outliers,max_price_zscore_1,max_price_zscore_2,max_price_zscore_3,max_price_IQR_70,max_price_IQR_80,max_price_IQR_90
item_category,,,,,,,
amulet,85547,231,462,690,11,11,23
belt,4050000,4830,4830,20000,6,11,23
boots,6249999,8100,8100,8100,6,11,24
bow,7699923,34500,34500,34500,6,11,24
chest,88923,360,728,1102,6,11,36
claw,847982,3311,8395,8395,6,11,24
dagger,13498,90,169,243,8,11,36
gloves,99999,364,719,1080,8,11,29
helmet,6400,60,114,165,6,11,24


In [82]:
compare_categories_df = compare_categories_df.drop(['max_price_outliers','max_price_IQR_70','max_price_IQR_80','max_price_IQR_90'],axis=1)

In [83]:
compare_categories_df

,max_price_zscore_1,max_price_zscore_2,max_price_zscore_3
item_category,,,
amulet,231.0,462.000,690.000
belt,4830.0,4830.000,20000.000
boots,8100.0,8100.000,8100.000
bow,34500.0,34500.000,34500.000
chest,360.0,728.914,1102.487
claw,3311.0,8395.865,8395.865
dagger,90.0,169.613,243.000
gloves,364.5,719.915,1080.000
helmet,60.0,114.000,165.000


## It becomes clear that weapons have lower count than armour here

### It also becomes clear that z_score_2 gives as better price_variance and price_max

In [ ]:
for key_item_category, type_of_df in dfs_per_category.items():
    print("count for category -{}- = {}".format(key_item_category,type_of_df['stats_zscore_2.csv'].loc['price_amount','count']))

In [103]:
for key_item_category, type_of_df in dfs_per_category.items():
    print("\n\nStats for {}".format(key_item_category))
    
    for df_category,df in type_of_df.items():
        
        if 'zscore' in df_category:
            print("\t\tPrice stats for {}".format(df_category))
            
            df.loc['price_amount',['variance','mean','max']]



Stats for AMULET
		Price stats for stats_zscore_1.csv


variance    185.140
mean          5.455
max         231.000
Name: price_amount, dtype: float64

		Price stats for stats_zscore_2.csv


variance    346.079
mean          5.945
max         462.000
Name: price_amount, dtype: float64

		Price stats for stats_zscore_3.csv


...

variance    42.575
mean         3.608
max         60.000
Name: price_amount, dtype: float64

		Price stats for stats_zscore_2.csv


variance     77.205
mean          4.071
max         110.000
Name: price_amount, dtype: float64

		Price stats for stats_zscore_3.csv


variance    129.349
mean          4.447
max         170.000
Name: price_amount, dtype: float64

In [140]:
helmet_df = dfs_per_category['chest']['stats_zscore_2.csv']
one_percent = helmet_df.loc['date_day','count']*0.01
# print("features with corr > 0.1 and <-0.1: {}".format(helmet_df.loc[abs(helmet_df['corr_kend_price'])>0.1]))
helmet_df.loc[abs(helmet_df['corr_kend_price'])>0.09].loc[helmet_df['count'] > one_percent]
helmet_df.loc[abs(helmet_df['corr_spear_price'])>0.09]
helmet_df['corr_spear_price'].nlargest(50)

,count,mean,std,min,25%,50%,75%,max,median,variance,corr_kend_price,corr_pears_price,corr_spear_price,percentage_to_rows
Unnamed: 0,,,,,,,,,,,,,,
Armour,70033,646.720,345.837,19.000,385.0,575.0,841.0,2634.000,575.0,119603.061,0.121,0.110,0.169,50.704
Energy Shield,64333,131.017,82.892,3.000,75.0,109.0,163.0,639.000,109.0,6871.007,0.102,0.136,0.141,46.577
Evasion Rating,57795,577.876,312.371,27.000,353.0,498.0,737.0,2534.000,498.0,97575.426,0.133,0.095,0.185,41.844
Quality,138121,2.643,6.232,0.000,0.0,0.0,0.0,30.000,0.0,38.844,0.147,0.255,0.174,100.000
corrupted,138121,0.175,0.380,0.000,0.0,0.0,0.0,1.000,0.0,0.144,0.236,0.045,0.270,100.000
date_day,138121,6.046,1.859,4.000,4.0,6.0,7.0,10.000,6.0,3.457,0.122,0.114,0.155,100.000
days_in_snapshot,138121,1.110,1.388,0.000,0.0,1.0,2.0,6.000,1.0,1.928,-0.096,-0.085,-0.119,100.000
ex_# to armour,31142,82.956,96.306,3.000,19.0,42.0,98.0,505.000,42.0,9274.844,0.108,0.067,0.151,22.547
ex_# to dexterity,18708,25.332,10.991,8.000,16.0,25.0,34.0,55.000,25.0,120.811,0.115,0.080,0.158,13.545


,count,mean,std,min,25%,50%,75%,max,median,variance,corr_kend_price,corr_pears_price,corr_spear_price,percentage_to_rows
Unnamed: 0,,,,,,,,,,,,,,
Armour,70033,646.720,345.837,19.000,385.00,575.00,841.00,2634.000,575.00,119603.061,0.121,0.110,0.169,50.704
Energy Shield,64333,131.017,82.892,3.000,75.00,109.00,163.00,639.000,109.00,6871.007,0.102,0.136,0.141,46.577
Evasion Rating,57795,577.876,312.371,27.000,353.00,498.00,737.00,2534.000,498.00,97575.426,0.133,0.095,0.185,41.844
Quality,138121,2.643,6.232,0.000,0.00,0.00,0.00,30.000,0.00,38.844,0.147,0.255,0.174,100.000
co_#% additional physical damage reduction while stationary,20,3.900,0.788,3.000,3.00,4.00,4.25,5.000,4.00,0.621,0.084,0.242,0.104,0.014
co_#% increased damage,66,45.652,3.145,40.000,43.00,46.00,49.00,50.000,46.00,9.892,0.157,0.093,0.204,0.048
co_#% increased maximum energy shield,22,4.818,0.795,4.000,4.00,5.00,5.00,6.000,5.00,0.632,0.182,0.431,0.215,0.016
co_#% increased maximum life,28,4.893,0.916,4.000,4.00,5.00,6.00,6.000,5.00,0.840,-0.192,-0.290,-0.242,0.020
co_#% increased spell damage,165,6.509,2.331,3.000,4.00,7.00,9.00,10.000,7.00,5.434,-0.101,-0.132,-0.129,0.119


Unnamed: 0
price_amount                                                    1.000
co_#% reduced fire damage taken                                 0.602
ex_minions have #% increased maximum life                       0.597
ex_#% chance to avoid lightning damage when hit                 0.464
co_#% reduced lightning damage taken                            0.388
sockets_number                                                  0.382
item_level                                                      0.369
ex_spells have #% to critical strike chance                     0.355
ex_attacks have #% to critical strike chance                    0.333
ex_#% chance to avoid cold damage when hit                      0.280
corrupted                                                       0.270
ex_# to maximum life                                            0.268
ex_#% increased dexterity                                       0.259
ex_socketed gems are supported by level # item rarity           0.231
co_#% inc

In [115]:
for key_item_category, type_of_df in dfs_per_category.items():
    print("\n\n{} in stats_zscore_2.csv".format(key_item_category.upper()))
    
    for df_category,df in type_of_df.items():
        
        if 'stats_zscore_2.csv' in df_category:
            #print("\t\tPrice stats for {}".format(df_category))
            print("number of features {}".format(df.shape[0]))
            print("number of features with corr > 0.1 and <-0.1: {}".format(df.loc[abs(df['corr_kend_price'])>0.1].loc[df['price_amount']].shape[0]))
            #df.loc['price_amount',['variance','mean','max']]



AMULET in stats_zscore_2.csv
number of features 170
number of features with corr > 0.1 and <-0.1: 44


BELT in stats_zscore_2.csv
number of features 99
number of features with corr > 0.1 and <-0.1: 31


BOOTS in stats_zscore_2.csv
number of features 106
number of features with corr > 0.1 and <-0.1: 37


BOW in stats_zscore_2.csv
number of features 106
number of features with corr > 0.1 and <-0.1: 47


CHEST in stats_zscore_2.csv
number of features 94
number of features with corr > 0.1 and <-0.1: 45


CLAW in stats_zscore_2.csv
number of features 109
number of features with corr > 0.1 and <-0.1: 65


DAGGER in stats_zscore_2.csv
number of features 131
number of features with corr > 0.1 and <-0.1: 55


GLOVES in stats_zscore_2.csv
number of features 179
number of features with corr > 0.1 and <-0.1: 53


HELMET in stats_zscore_2.csv
number of features 570
number of features with corr > 0.1 and <-0.1: 51


JEWELS in stats_zscore_2.csv
number of features 150
number of features with corr >

In [ ]:
columns = ['item_category','stats_zscore_1.csv','stats_zscore_2.csv','stats_zscore_3.csv']

compare_categories_df =  pd.DataFrame(columns=columns)

for key_item_category, type_of_df in dfs_per_category.items():
    
    category_dict = {'item_category':key_item_category}
    
    for df_category,df in type_of_df.items():
        category_dict[df_category] =df.loc['price_amount','max']
    compare_categories_df = compare_categories_df.append(category_dict,ignore_index=True)

compare_categories_df = compare_categories_df.set_index('item_category')